In [1]:
import numpy as np
import pandas as pd

In [2]:
# - 피봇 테이블(Pivot Table)
# 피봇테이블이란 데이터 열 중에서 두 개를 키(key)로 사용해서 데이터를 선택하는 방법을 말한다
# Pandas는 피봇테이블을 만들기 위한 pivot메서드를 제공한다. 첫 번째 인수로는 행 인덱스로 사용할
# 열이름,두번째 인수로는 열 인덱스로 사용할 열 이름, 그리고, 마지막으로 데이터로 사용할 열이름을
# 넣는다.
# Pandas는 지정된 두 열을 각각 행 인덱스와 열 인덱스로 바꾼 후, 행 인덱스의 라벨 값이 첫번째 키의
# 값과 같고, 열 인덱스의 라벨 값이 두번째 키의 값과 같은 데이터를 찾아서 해당 칸에 넣는다. 만약
# 주어진 데이터가 존재하지 않으면 해당 칸은 NaN 값을 넣는다.

data = {
    "도시":["서울", "서울", "서울", "부산", "부산", "부산", "인천", "인천"],
    "연도":["2015", "2010", "2005", "2015", "2010","2005", "2015", "2010"],
    "인구":[9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 2632035],
    "지역": ["수도권", "수도권", "수도권", "경상권", "경상권", "경상권", "수도권", "수도권"]
}
columns = ["도시","연도", "인구", "지역"]
df1 = pd.DataFrame(data, columns=columns)
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,2632035,수도권


In [3]:
df1.pivot("도시", "연도", "인구")

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,2632035.0,2890451.0


In [4]:
# 사실 피봇 테이블은 다음과 같이 set_index명령과 unstack명령을 사용해서 만들 수도 있다.
df1.set_index(["도시","연도"])[["인구"]].unstack()

인구                      
연도       2005       2010       2015
도시                                 
부산  3512547.0  3393191.0  3448737.0
서울  9762546.0  9631482.0  9904312.0
인천        NaN  2632035.0  2890451.0

In [5]:
# 행 인덱스와 열 인덱스는 하나의 데이터를 찾는 키(key)의 역할을 한다. 즉. 이 값으로 데이터가
# 유일하게 결정되어야 한다.
# df1.pivot("지역", "연도", "인구")  # 에러: (수도권, 2015)가 중복

In [6]:
# 그룹분석
# 그룹분석은 피봇 테이블과 달리 키에 의해 결정되는 데이터가 여러 개가 있어도 괜찮다.
# 대신 주어진 연산을 통해  그룹 데이터의 대표 값을 정하게 된다.
np.random.seed(0)
df2 = pd.DataFrame({
    'key1': ['A', 'A', 'B', 'B', 'A'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data1': [1, 2, 3, 4, 5],
    'data2': [10, 20, 30, 40, 50]
})
df2

,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


In [7]:
df2.groupby(df2.key1).sum()

,data1,data2
key1,,
A,8,80
B,7,70


In [8]:
df2.data1.groupby(df2.key1).sum()

key1
A    8
B    7
Name: data1, dtype: int64

In [9]:
df2.groupby(df2.key1).sum()["data1"]

key1
A    8
B    7
Name: data1, dtype: int64

In [10]:
# 분석하고자 하는 키가 복수이면 리스트를 사용한다.
df2.data1.groupby([df2.key1, df2.key2]).sum()

key1  key2
A     one     6
      two     2
B     one     3
      two     4
Name: data1, dtype: int64

In [11]:
# 이 결과를 unstack 명령으로 피봇테이블 형태로 만들면 더 보기기 좋아진다.
df2.data1.groupby([df2["key1"], df2["key2"]]).sum().unstack("key2")

key2,one,two
key1,,
A,6,2
B,3,4


In [12]:
df1["인구"].groupby([df1["지역"], df1["연도"]]).sum().unstack()

연도,2005,2010,2015
지역,,,
경상권,3512547,3393191,3448737
수도권,9762546,12263517,12794763


In [13]:
# 다음 데이터는 150송이의 붓꽃에 대해 붓꽃 종별 데이터. (seaborn 패키지 설치 요)
import seaborn as sns
iris = sns.load_dataset("iris")

# 각 붓꽃 종별로 가장 큰 값과 가장 작은 값의 비율을 구해보자.
def peak_to_peak_ratio(x):
    return x.max() / x.min()

In [14]:
iris.groupby(iris.species).agg(peak_to_peak_ratio)

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,1.348837,1.913043,1.900000,6.000000
versicolor,1.428571,1.700000,1.700000,1.800000
virginica,1.612245,1.727273,1.533333,1.785714


In [15]:
# describe를 사용하면 다양한 기술 통계 값을 구할 수 있다.
# 그룹별로 하나의 스칼라 값이 아니라 하나의 데이터프레임이 생성된다.
iris.groupby(iris.species).describe().T

species                setosa  versicolor  virginica
petal_length count  50.000000   50.000000  50.000000
             mean    1.462000    4.260000   5.552000
             std     0.173664    0.469911   0.551895
             min     1.000000    3.000000   4.500000
             25%     1.400000    4.000000   5.100000
             50%     1.500000    4.350000   5.550000
             75%     1.575000    4.600000   5.875000
             max     1.900000    5.100000   6.900000
petal_width  count  50.000000   50.000000  50.000000
             mean    0.246000    1.326000   2.026000
             std     0.105386    0.197753   0.274650
             min     0.100000    1.000000   1.400000
             25%     0.200000    1.200000   1.800000
             50%     0.200000    1.300000   2.000000
             75%     0.300000    1.500000   2.300000
             max     0.600000    1.800000   2.500000
sepal_length count  50.000000   50.000000  50.000000
             mean    5.006000    5.936000   6.588000
             std     0.352490    0.516171   0.635880
             min     4.300000    4.900000   4.900000
             25%     4.800000    5.600000   6.225000
             50%     5.000000    5.900000   6.500000
             75%     5.200000    6.300000   6.900000
             max     5.800000    7.000000   7.900000
sepal_width  count  50.000000   50.000000  50.000000
             mean    3.428000    2.770000   2.974000
             std     0.379064    0.313798   0.322497
             min     2.300000    2.000000   2.200000
             25%     3.200000    2.525000   2.800000
             50%     3.400000    2.800000   3.000000
             75%     3.675000    3.000000   3.175000
             max     4.400000    3.400000   3.800000

In [16]:
# apply. 각 붓꽃 종별로 가장 꽃잎 길이가 큰 3개의 데이터
def max3(df):
    return df.sort_values(by="petal_length")[:3]
iris.groupby(iris.species).apply(max3)

sepal_length  sepal_width  petal_length  petal_width  \
species                                                                
setosa     22            4.6          3.6           1.0          0.2   
           13            4.3          3.0           1.1          0.1   
           14            5.8          4.0           1.2          0.2   
versicolor 98            5.1          2.5           3.0          1.1   
           93            5.0          2.3           3.3          1.0   
           57            4.9          2.4           3.3          1.0   
virginica  106           4.9          2.5           4.5          1.7   
           126           6.2          2.8           4.8          1.8   
           138           6.0          3.0           4.8          1.8   

                   species  
species                     
setosa     22       setosa  
           13       setosa  
           14       setosa  
versicolor 98   versicolor  
           93   versicolor  
           57   versicolor  
virginica  106   virginica  
           126   virginica  
           138   virginica

In [17]:
# transform
def q3cut(s):
    return pd.qcut(s, 3, labels=["소", "중", "대"])
iris["petal_length_class"] = iris.groupby(iris.species)["petal_length"].transform(q3cut)
iris[["petal_length", "petal_length_class"]].tail(10)

,petal_length,petal_length_class
140,5.6,중
141,5.1,소
142,5.1,소
143,5.9,대
144,5.7,중
145,5.2,소
146,5.0,소
147,5.2,소
148,5.4,중
149,5.1,소


In [18]:
# pivot_table: groupby 명령의 결과에 unstack을 자동적용하여 2차원적 형태로 변형.
# pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, margins_name='All')
# pivot_table을 메서드로 사용할 때는 객체 자차게 데이터가 되므로 data인수가 필요하지 않다.
df2.pivot_table("data1", "key1", "key2")

key2,one,two
key1,,
A,3,2
B,3,4


In [19]:
df2.pivot_table("data1", "key1", "key2", margins=True, margins_name="합계")

key2,one,two,합계
key1,,,
A,3,2,2.666667
B,3,4,3.500000
합계,3,3,3.000000


In [20]:
# 행 인덱스나 열 인덱스에 리스트를 넣으면 다중 인덱스 테이블 생성
df2.pivot_table("data1", index=["key1", "key2"])

data1
key1 key2       
A    one       3
     two       2
B    one       3
     two       4

In [22]:
# TIP 데이터 예제
tips = sns.load_dataset("tips")
tips.tail()

,total_bill,tip,sex,smoker,day,time,size
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2
243,18.78,3.00,Female,No,Thur,Dinner,2


In [41]:
tips['tip_pct'] = tips['tip']/tips['total_bill']
tips.tail()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204
243,18.78,3.00,Female,No,Thur,Dinner,2,0.159744


In [42]:
tips.describe()

,total_bill,tip,size,tip_pct
count,244.000000,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672,0.160803
std,8.902412,1.383638,0.951100,0.061072
min,3.070000,1.000000,1.000000,0.035638
25%,13.347500,2.000000,2.000000,0.129127
50%,17.795000,2.900000,2.000000,0.154770
75%,24.127500,3.562500,3.000000,0.191475
max,50.810000,10.000000,6.000000,0.710345


In [27]:
tips.groupby(["sex", "smoker"]).size()

sex     smoker
Male    Yes       60
        No        97
Female  Yes       33
        No        54
dtype: int64

In [43]:
tips.pivot_table("tip_pct", "sex", "smoker", aggfunc="count", margins=True)

smoker,Yes,No,All
sex,,,
Male,60,97,157
Female,33,54,87
All,93,151,244


In [44]:
tips.groupby("sex")[["tip_pct"]].mean()

,tip_pct
sex,
Male,0.157651
Female,0.166491


In [45]:
tips.groupby("smoker")[["tip_pct"]].mean()

,tip_pct
smoker,
Yes,0.163196
No,0.159328


In [46]:
# pivot table을 사용할 수 있다.
tips.pivot_table("tip_pct","sex")

,tip_pct
sex,
Male,0.157651
Female,0.166491


In [49]:
tips.pivot_table("tip_pct", ["sex", "smoker"])

tip_pct
sex    smoker          
Male   Yes     0.152771
       No      0.160669
Female Yes     0.182150
       No      0.156921

In [48]:
tips.pivot_table("tip_pct", "sex", "smoker")

smoker,Yes,No
sex,,
Male,0.152771,0.160669
Female,0.182150,0.156921


In [51]:
tips.groupby("sex")[["tip_pct"]].describe()

tip_pct                                                              \
         count      mean       std       min       25%       50%       75%   
sex                                                                          
Male     157.0  0.157651  0.064778  0.035638  0.121389  0.153492  0.186240   
Female    87.0  0.166491  0.053632  0.056433  0.140416  0.155581  0.194266   

                  
             max  
sex               
Male    0.710345  
Female  0.416667

In [52]:
tips.groupby("smoker")[["tip_pct"]].describe()

tip_pct                                                              \
         count      mean       std       min       25%       50%       75%   
smoker                                                                       
Yes       93.0  0.163196  0.085119  0.035638  0.106771  0.153846  0.195059   
No       151.0  0.159328  0.039910  0.056797  0.136906  0.155625  0.185014   

                  
             max  
smoker            
Yes     0.710345  
No      0.291990

In [53]:
tips.groupby(["sex","smoker"])[["tip_pct"]].describe()

tip_pct                                                    \
                count      mean       std       min       25%       50%   
sex    smoker                                                             
Male   Yes       60.0  0.152771  0.090588  0.035638  0.101845  0.141015   
       No        97.0  0.160669  0.041849  0.071804  0.131810  0.157604   
Female Yes       33.0  0.182150  0.071595  0.056433  0.152439  0.173913   
       No        54.0  0.156921  0.036421  0.056797  0.139708  0.149691   

                                   
                    75%       max  
sex    smoker                      
Male   Yes     0.191697  0.710345  
       No      0.186220  0.291990  
Female Yes     0.198216  0.416667  
       No      0.181630  0.252672

In [54]:
# 각 그룹에서 가장 많은 팁과 가장 적은 팁의 차이
def peak_to_peak(x):
    return x.max() - x.min()

tips.groupby(["sex", "smoker"])[["tip"]].agg(peak_to_peak)

tip
sex    smoker      
Male   Yes     9.00
       No      7.75
Female Yes     5.50
       No      4.20

In [62]:
# 여러가지 그룹 연산을 동시에 하고 싶다면, 리스트 이용
# tips.groupby(["sex","smoker"]).agg(["mean", peak_to_peak])
tips.groupby(["sex","smoker"]).agg(["mean", peak_to_peak])[["total_bill"]]

total_bill             
                    mean peak_to_peak
sex    smoker                        
Male   Yes     22.284500        43.56
       No      19.791237        40.82
Female Yes     17.977879        41.23
       No      18.105185        28.58

In [58]:
#  만약 열마다 다른 연산 ->  열라벨 연산과 연산 이름(또는 함수)를 딕셔너리로 넣는다.
tips.groupby(["sex","smoker"]).agg(
    {'tip_pct': 'mean', 'total_bill': peak_to_peak}
)

tip_pct  total_bill
sex    smoker                      
Male   Yes     0.152771       43.56
       No      0.160669       40.82
Female Yes     0.182150       41.23
       No      0.156921       28.58

In [63]:
# 더 복잡한 분석
tips.pivot_table(['tip_pct','size'],['sex','day'], 'smoker')

size             tip_pct          
smoker            Yes        No       Yes        No
sex    day                                         
Male   Thur  2.300000  2.500000  0.164417  0.165706
       Fri   2.125000  2.000000  0.144730  0.138005
       Sat   2.629630  2.656250  0.139067  0.162132
       Sun   2.600000  2.883721  0.173964  0.158291
Female Thur  2.428571  2.480000  0.163073  0.155971
       Fri   2.000000  2.500000  0.209129  0.165296
       Sat   2.200000  2.307692  0.163817  0.147993
       Sun   2.500000  3.071429  0.237075  0.165710

In [64]:
tips.pivot_table('size', ['time', 'sex','smoker'], 'day', aggfunc='sum', fill_value=0)

day                   Thur  Fri  Sat  Sun
time   sex    smoker                     
Lunch  Male   Yes       23    5    0    0
              No        50    0    0    0
       Female Yes       17    6    0    0
              No        60    3    0    0
Dinner Male   Yes        0   12   71   39
              No         0    4   85  124
       Female Yes        0    8   33   10
              No         2    2   30   43